In [1]:
import pandas as pd

TRAIN = '/kaggle/input/digit-recognizer/train.csv'
train_df = pd.read_csv(filepath_or_buffer=TRAIN, )
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
from warnings import filterwarnings
from plotly import express
filterwarnings(action='ignore', category=UserWarning)

express.histogram(data_frame=train_df, x='label',).show()
express.pie(data_frame=train_df, names='label', color='label').show()

In [3]:
from arrow import now
from umap import UMAP

time_start = now()
columns = [column for column in train_df.columns if column.startswith('pixel')]
umap = UMAP(random_state=2024, verbose=True, n_jobs=1, low_memory=False, n_epochs=500)
train_df[['x', 'y']] = pd.DataFrame(data=umap.fit_transform(X=train_df[columns]))
express.scatter(data_frame=train_df, x='x', y='y', color='label').show()
print('done with UMAP in {}'.format(now() - time_start))

2024-03-28 17:09:09.341498: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 17:09:09.341697: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28 17:09:09.526156: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


UMAP(low_memory=False, n_epochs=500, n_jobs=1, random_state=2024, verbose=True)
Thu Mar 28 17:09:26 2024 Construct fuzzy simplicial set
Thu Mar 28 17:09:26 2024 Finding Nearest Neighbors
Thu Mar 28 17:09:26 2024 Building RP forest with 15 trees
Thu Mar 28 17:09:37 2024 NN descent for 15 iterations
	 1  /  15
	 2  /  15
	 3  /  15
	 4  /  15
	 5  /  15
	Stopping threshold met -- exiting after 5 iterations
Thu Mar 28 17:10:09 2024 Finished Nearest Neighbor Search
Thu Mar 28 17:10:13 2024 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Thu Mar 28 17:11:34 2024 Finished embedding


done with UMAP in 0:02:08.725592


Four of our digits are for the most part in isolated clusters; the other six digits are in clusters that have regions of ambiguity.

Let's separate out a validation set and use logistic regression to set a baseline for our model.

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from arrow import now

time_start = now()
target = 'label'
X_train, X_test, y_train, y_test = train_test_split(train_df[columns], train_df[target], test_size=0.1, random_state=2024, stratify=train_df[target])

regression = LogisticRegression(max_iter=500, tol=1e-6, verbose=1)
regression.fit(X=X_train, y=y_train)
print('fit complete after {} iterations.'.format(regression.n_iter_[0]))
print('accuracy: {:5.4f}'.format(regression.score(X=X_test, y=y_test)))
print('done in {}'.format(now() - time_start))

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         7850     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  8.70377D+04    |proj g|=  6.09411D+05


 This problem is unconstrained.



At iterate   50    f=  9.45550D+03    |proj g|=  9.89739D+03

At iterate  100    f=  8.31421D+03    |proj g|=  1.04489D+04

At iterate  150    f=  7.80817D+03    |proj g|=  4.56246D+03

At iterate  200    f=  7.48628D+03    |proj g|=  3.75506D+03

At iterate  250    f=  7.25377D+03    |proj g|=  2.81633D+03

At iterate  300    f=  7.08808D+03    |proj g|=  2.84372D+03

At iterate  350    f=  6.96311D+03    |proj g|=  1.92796D+03

At iterate  400    f=  6.86079D+03    |proj g|=  2.93551D+03

At iterate  450    f=  6.78118D+03    |proj g|=  3.76133D+03

At iterate  500    f=  6.71204D+03    |proj g|=  1.70637D+03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tn

In [5]:
from arrow import now
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

MODEL = {
    'Neural Net 1.0': MLPClassifier(alpha=1, max_iter=2000, random_state=2024),
}

for n_neighbors in range(2, 12):
    MODEL['{} Nearest Neighbors'.format(n_neighbors)] = KNeighborsClassifier(n_neighbors=n_neighbors)

for n_estimators in range(5, 100, 5):
    MODEL['{} estimator Random Forest'.format(n_estimators)] = RandomForestClassifier(n_estimators=n_estimators,  random_state=2024)

result = []
for name, clf in MODEL.items():
    time_start = now()
    clf.fit(X=X_train, y=y_train)
    score = clf.score(X=X_test, y=y_test)
    result.append((score, name))
    print('{:5.4f} {} {}'.format(score, now() - time_start, name))
result = sorted(result, key=lambda x: x[0], reverse=True)
print('best: {} {}'.format(result[0][0], result[0][1]))

0.9569 0:02:20.449022 Neural Net 1.0
0.9631 0:00:07.599019 2 Nearest Neighbors
0.9693 0:00:07.913540 3 Nearest Neighbors
0.9686 0:00:08.274705 4 Nearest Neighbors
0.9683 0:00:07.868996 5 Nearest Neighbors
0.9674 0:00:08.121704 6 Nearest Neighbors
0.9671 0:00:08.104046 7 Nearest Neighbors
0.9676 0:00:08.095837 8 Nearest Neighbors
0.9669 0:00:07.740358 9 Nearest Neighbors
0.9667 0:00:07.912488 10 Nearest Neighbors
0.9657 0:00:08.035423 11 Nearest Neighbors
0.9088 0:00:01.989656 5 estimator Random Forest
0.9390 0:00:03.952475 10 estimator Random Forest
0.9479 0:00:05.715958 15 estimator Random Forest
0.9538 0:00:07.682308 20 estimator Random Forest
0.9602 0:00:09.470253 25 estimator Random Forest
0.9621 0:00:11.309767 30 estimator Random Forest
0.9633 0:00:13.244983 35 estimator Random Forest
0.9638 0:00:14.839929 40 estimator Random Forest
0.9624 0:00:16.743778 45 estimator Random Forest
0.9612 0:00:18.585775 50 estimator Random Forest
0.9605 0:00:20.636921 55 estimator Random Forest
0.9

Let's make our submission.

In [6]:
TEST = '/kaggle/input/digit-recognizer/test.csv'

best = KNeighborsClassifier(n_neighbors=3)
best.fit(X=train_df[columns], y=train_df[target])
test_df = pd.read_csv(filepath_or_buffer=TEST, )

result_df = pd.DataFrame(data=best.predict(X=test_df[columns])).reset_index()
result_df.columns = ['ImageId', 'Label']
result_df.to_csv(path_or_buf='/kaggle/working/submission.csv', index=False)